In [16]:
using Distributions
using HCubature

In [33]:
function get_collision_probability_edge(n1,t1,n2,t2,t_edge,lambda)
    #Count edge conflicts quickly to elimininate cases with zeros
    res = @timed(count_edge_conflicts(n1,t1,n2,t2,t_edge,lambda;num_particles=1000))
    preliminary_count = res[1]
    dtcount = res[2]
    if preliminary_count < 0.0001
        return 0.0,0.0,dt_count
    else
        return integrate_edge_with_box(n1,t1,n2,t2,t_edge,lambda,dtcount)
    end
end

function integrate_edge_with_box(n1,t1,n2,t2,t_edge,lambda,dtcount)
    function h(x)
        e2 = x[1]
        density = (cdf(Gamma(n2,lambda),max(0,(e2+t2-t1+t_edge))) -  cdf(Gamma(n2,lambda),max(0,(e2+t2-t1-t_edge))) ) * pdf(Gamma(n1,lambda), e2)
        return density
    end

    a = [0.0]
    b = [200.0]

    res1 = @timed(hcubature(h,a,b,maxevals=10^8))
    C,err = res1[1]
    dtint = res1[2] #time spent performing integration

    return C, err, dtint + dtcount

end

function count_edge_conflicts(n1,t1,n2,t2,t_edge,lambda;num_particles=10000)
    """Monte Carlo simulation"""
    EA1 = rand(Gamma(n1,lambda),num_particles)
    EA2 = rand(Gamma(n2,lambda),num_particles)

    r1_arrivals = t1 .+ EA1
    r1_departures = r1_arrivals .+ t_edge
    r2_arrivals = t2 .+ EA2
    r2_departures = r2_arrivals .+ t_edge

    num_conflicts =length(findall(((r2_departures-r1_arrivals).>0) .& ((r1_departures-r2_arrivals).>0)   ))

    return num_conflicts/num_particles
end

count_edge_conflicts (generic function with 1 method)

In [34]:
n1 = 3
n2 = 15
t1 = 5.0
t2 = 1.0
t_edge = 1.0
lambda = 0.2

0.2

In [35]:
count_edge_conflicts(n1,t1,n2,t2,t_edge,lambda;num_particles=10000)

0.2204

In [36]:
get_collision_probability_edge(n1,t1,n2,t2,t_edge,lambda)

(2.5996695069393797e-9, 1.5214620183295645e-18, 0.023751438)